# Importing Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append("../source/")
sys.path.append("../")

import paths

from dotenv import load_dotenv

load_dotenv("../.env")

import pinecone
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain

REPLICATE_API_TOKEN = os.environ["REPLICATE_API_TOKEN"]
PINECONE_API_TOKEN = os.environ["PINECONE_API_TOKEN"]

pinecone.init(api_key = PINECONE_API_TOKEN, environment = "gcp-starter")

/home/zero/.cache/pypoetry/virtualenvs/src-TO89gsno-py3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [16]:
#Fetching all .txt Files in the data/transformed/ Directory

Documents = []

for x in os.listdir(paths.TRANSFORMED_DATA_DIR):
    if x.endswith(".txt"):
        Loader = TextLoader(paths.TRANSFORMED_DATA_DIR / x)
        Document = Loader.load()
        
        TextSplitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
        Text = TextSplitter.split_documents(Document)
        Embeddings = HuggingFaceEmbeddings()
        
        #Loading Documents into the Vector Database

        IndexName = "centralbanksllm"
        Index = pinecone.Index(IndexName)
        VectorDB = Pinecone.from_documents(Text, Embeddings, index_name = IndexName, )
        
    break

ApiValueError: Unable to prepare type PosixPath for serialization

In [3]:
        #Loading Documents into the Vector Database

        IndexName = "centralbanksllm"
        Index = pinecone.Index(IndexName)
        VectorDB = Pinecone.from_texts(Text, Embeddings, index_name = IndexName, )
        
    break

ApiValueError: Unable to prepare type PosixPath for serialization

In [ ]:
#Loading Documents into the Vector Database

IndexName = "centralbanksllm"
Index = pinecone.Index(IndexName)
VectorDB = Pinecone.from_documents(Texts, Embeddings, index_name = IndexName)

In [ ]:
LLM = Replicate(model = "a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5", input = {"temperature": 0.75, "max_length": 3000})

In [ ]:
QA_Chain = ConversationalRetrievalChain.from_llm(LLM, VectorDB.as_retriever(search_kwargs = {'k': 2}), return_source_documents = True)

In [ ]:
ChatHistory = []

while True:
    Query = input('Prompt: ')
    
    if Query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
        
    Result = QA_Chain({'question': Query, 'chat_history': ChatHistory})
    print('Answer: ' + Result['answer'] + '\n')
    ChatHistory.append((Query, Result['answer']))